In [1]:
!git clone "https://github.com/nhimxu00/face-anti-spoofing.git"

Cloning into 'face-anti-spoofing'...
remote: Enumerating objects: 929, done.
remote: Counting objects: 100% (929/929), done.
remote: Compressing objects: 100% (919/919), done.
remote: Total 929 (delta 9), reused 929 (delta 9), pack-reused 0
Receiving objects: 100% (929/929), 40.60 MiB | 19.27 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [2]:
%cd /content/face-anti-spoofing

/content/face-anti-spoofing


In [3]:
!ls -l

total 22696
drwxr-xr-x 3 root root     4096 Mar 21 03:43  config
drwxr-xr-x 3 root root     4096 Mar 21 03:43  data
drwxr-xr-x 3 root root     4096 Mar 21 03:43  datasets
drwxr-xr-x 2 root root     4096 Mar 21 03:43  docs
drwxr-xr-x 2 root root     4096 Mar 21 03:43  evaluation
drwxr-xr-x 3 root root     4096 Mar 21 03:43  experiments
-rw-r--r-- 1 root root 14497985 Mar 21 03:43  face.mp4
drwxr-xr-x 2 root root     4096 Mar 21 03:43  inference
drwxr-xr-x 3 root root     4096 Mar 21 03:43  models
-rw-r--r-- 1 root root  1636772 Mar 21 03:43 'mtcnn face.jpg'
-rw-r--r-- 1 root root       72 Mar 21 03:43  requirements.txt
drwxr-xr-x 2 root root     4096 Mar 21 03:43  spliting_codes
-rw-r--r-- 1 root root  1162139 Mar 21 03:43  test2.jpg
-rw-r--r-- 1 root root  5885774 Mar 21 03:43  test2.png
drwxr-xr-x 3 root root     4096 Mar 21 03:43  trainer
-rw-r--r-- 1 root root     2889 Mar 21 03:43  train.py
drwxr-xr-x 3 root root     4096 Mar 21 03:43  utils


In [4]:
!pip install --upgrade -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.1 MB/s eta 0:00:00


In [5]:
import wandb
wandb.login(key='d65b790638e3fd6cf9f52a2ff3571e0e2b6a45ff')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import os
import torch
from torchvision import transforms, models
from torch.utils.tensorboard import SummaryWriter
from datasets.FASDataset import FASDataset
from utils.transform import RandomGammaCorrection
from utils.utils import read_cfg, get_optimizer, get_device, build_network
from trainer.FASTrainer import FASTrainer
from models.loss import DepthLoss
from torch.optim.lr_scheduler import StepLR
from datetime import datetime

In [7]:
cfg = read_cfg(cfg_file="config/CDCNpp_adam_lr1e-3.yaml")

device = get_device(cfg)

network = build_network(cfg)

optimizer = get_optimizer(cfg, network)

lr_scheduler = StepLR(optimizer=optimizer, step_size=30, gamma=0.1)

criterion = DepthLoss(device=device)

writer = SummaryWriter(cfg['log_dir'])

dump_input = torch.randn((1, 3, cfg['model']['input_size'][0], cfg['model']['input_size'][1]))

writer.add_graph(network, dump_input)

In [8]:
run = wandb.init(project = "TEST_1",
                 name = f"Run_{datetime.now().strftime('%d%m%Y%H%M%S')}", 
                 notes = "increase learning rate",
                 tags = [],
                 config = cfg,
)

wandb: Currently logged in as: minhxoanxu. Use `wandb login --relogin` to force relogin


In [9]:
train_transform = transforms.Compose([
    RandomGammaCorrection(max_gamma=cfg['dataset']['augmentation']['gamma_correction'][1],
                            min_gamma=cfg['dataset']['augmentation']['gamma_correction'][0]),
    transforms.RandomResizedCrop(cfg['model']['input_size'][0]),
    # transforms.ColorJitter(
    #     brightness=cfg['dataset']['augmentation']['brightness'],
    #     contrast=cfg['dataset']['augmentation']['contrast'],
    #     saturation=cfg['dataset']['augmentation']['saturation'],
    #     hue=cfg['dataset']['augmentation']['hue']
    # ),
    transforms.RandomRotation(cfg['dataset']['augmentation']['rotation_range']),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(cfg['model']['input_size']),
    transforms.ToTensor(),
    transforms.Normalize(cfg['dataset']['mean'], cfg['dataset']['sigma'])
])

val_transform = transforms.Compose([
    transforms.Resize(cfg['model']['input_size']),
    transforms.ToTensor(),
    transforms.Normalize(cfg['dataset']['mean'], cfg['dataset']['sigma'])
])

trainset = FASDataset(
    root_dir=cfg['dataset']['root'],
    csv_file=cfg['dataset']['train_set'],
    depth_map_size=cfg['model']['depth_map_size'],
    transform=train_transform,
    smoothing=cfg['train']['smoothing']
)

valset = FASDataset(
    root_dir=cfg['dataset']['root'],
    csv_file=cfg['dataset']['val_set'],
    depth_map_size=cfg['model']['depth_map_size'],
    transform=val_transform,
    smoothing=cfg['train']['smoothing']
)

trainloader = torch.utils.data.DataLoader(
    dataset=trainset,
    batch_size=cfg['train']['batch_size'],
    shuffle=True,
    num_workers=2
)

valloader = torch.utils.data.DataLoader(
    dataset=valset,
    batch_size=cfg['val']['batch_size'],
    shuffle=True,
    num_workers=2
)

In [ ]:
trainer = FASTrainer(
    cfg=cfg, 
    network=network,
    optimizer=optimizer,
    criterion=criterion,
    lr_scheduler=lr_scheduler,
    device=device,
    trainloader=trainloader,
    valloader=valloader,
    writer=writer
)

trainer.train()

writer.close()